In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
engine = sqlite3.connect('./src/data.db')
catalyst = pd.read_sql('SELECT * FROM catalysts', engine)

In [6]:
catalyst = catalyst.sort_values(by=['date']).reset_index(drop=True)
catalyst

,ticker,disease,stage,date,catalyst
0,SPPI,Non-Hodgkin’s lymphoma,Approved,2009-09-04 00:00:00,"Approved September 4, 2009."
1,JAZZ,Fibromyalgia,CRL,2010-10-11 00:00:00,"CRL received October 11, 2010."
2,ASRT,Postherpetic neuralgia - shingles,Approved,2011-01-28 00:00:00,"Approved January 28, 2011."
3,ASRT,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."
4,SPPI,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."
...,...,...,...,...,...
3910,AZN,Chronic Kidney Disease (CKD),Phase 2b,2023-11-03 00:00:00,Phase 2b showed statistically significant and ...
3911,MOLN,Solid Tumors,Phase 1,2023-11-03 00:00:00,Phase 1 data presented at SITC continues to de...
3912,INKT,Solid Tumors,Phase 1,2023-11-03 00:00:00,Phase 1 trial of agenT-797 alone or in combina...
3913,REGN,Various cancers,Phase 1,2023-11-04 00:00:00,Phase 1 safety data reported that a total of 1...


In [7]:
train_size = 0.8
row_len = int(train_size * catalyst.shape[0])
train = catalyst.iloc[:row_len, :]
test = catalyst.iloc[row_len:, :]
print(train.shape)
print(test.shape)
print(catalyst.shape)

(3132, 5)
(783, 5)
(3915, 5)


In [8]:
train.head()

,ticker,disease,stage,date,catalyst
0,SPPI,Non-Hodgkin’s lymphoma,Approved,2009-09-04 00:00:00,"Approved September 4, 2009."
1,JAZZ,Fibromyalgia,CRL,2010-10-11 00:00:00,"CRL received October 11, 2010."
2,ASRT,Postherpetic neuralgia - shingles,Approved,2011-01-28 00:00:00,"Approved January 28, 2011."
3,ASRT,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."
4,SPPI,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."


In [9]:
test.head()

,ticker,disease,stage,date,catalyst
3132,LLY,Obesity,Phase 3,2023-06-24 00:00:00,Phase 3 efficacy and safety data shared at ADA...
3133,EWTX,Becker muscular dystrophy (BMD),Phase 1b,2023-06-26 00:00:00,Phase 1b 12-month study supported the hypothes...
3134,TERN,Obesity,Phase 1a,2023-06-26 00:00:00,Preclinical data demonstrated the ability of T...
3135,LLY,Obesity,Phase 2,2023-06-26 00:00:00,Phase 2 24-week data met the primary endpoint ...
3136,ACET,Non-Hodgkin’s lymphoma,Phase 1,2023-06-26 00:00:00,Phase 1 safety and efficacy data demonstrated ...


In [26]:
train.to_csv("train.csv", index=0)
test.to_csv("test.csv", index=0)
catalyst.to_csv("catalyst.csv", index=0)

In [13]:
from huggingface_hub import login, logout

In [14]:
login()

In [17]:
from datasets import load_dataset

In [27]:
dataset = load_dataset("roymgabriel/BioPharma")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ticker', 'disease', 'stage', 'date', 'catalyst'],
        num_rows: 3132
    })
    test: Dataset({
        features: ['ticker', 'disease', 'stage', 'date', 'catalyst'],
        num_rows: 783
    })
})